<a href="https://colab.research.google.com/github/zhuojg/style_classifier/blob/master/style_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from torchvision import models
import torch
import torch.nn as nn
import datetime
from torchvision import transforms, datasets
import tqdm
from torch.optim import SGD
import torch.utils.data.dataset
from torch.autograd import Variable
from torch.nn import BCELoss, CrossEntropyLoss
import numpy as np
import math
from PIL import Image, ImageFile
from google.colab import drive
import os
ImageFile.LOAD_TRUNCATED_IMAGES = True


class StyleClassifier(nn.Module):
    def __init__(self):
        super(StyleClassifier, self).__init__()
        self.pre_trained = models.vgg16(pretrained=True)
        self.cnn = nn.Sequential(*list(self.pre_trained.children())[0])
        self.conv_dr = nn.Conv2d(512, 128, 1)
        self.fc = nn.Linear(in_features=128 * 128, out_features=1024)
        self.fc_1 = nn.Linear(in_features=1024, out_features=4)
#         self.fc = nn.Linear(in_features=1000, out_features=1)
        self.sig = nn.Sigmoid()

    def get_style_gram(self, style_features):
        style_features = style_features.view(-1, 7 * 7, 128)
        style_features_t = style_features.permute(0, 2, 1)
        grams = []
        for i in range(len(style_features)):
#             normalize the gram matrix
#             temp_result = np.matmul(style_features_t[i].cpu().detach().numpy(), style_features[i].cpu().detach().numpy())
#             m = np.mean(temp_result, axis=0)
#             std = np.std(temp_result, axis=0)

#             grams.append((torch.matmul(style_features_t[i], style_features[i]) - torch.from_numpy(m).cuda()) / torch.from_numpy(std).cuda())
            grams.append(torch.matmul(style_features_t[i], style_features[i]))

        grams = torch.stack(grams, 0)
        grams = grams.view(-1, 128 * 128)

        return grams

    def forward(self, x):
        h = x
        h = self.cnn(h)
        h = self.conv_dr(h)
        
        h = h.permute(0, 2, 3, 1)
        h = self.get_style_gram(h)
        
        h = self.fc(h)
        h = self.fc_1(h)

        return h


class ImageLoader(torch.utils.data.dataset.Dataset):
    def __init__(self, data_path, txt_path, is_train):
#         self.data = []
#         self.data_path = data_path
        self.is_train = is_train
#         with open(txt_path, 'r') as f:
#             for line in f.readlines():
#                 if line.split('/')[0] == 'modern':
#                     label = 1
#                 else:
#                     label = 0
#                 self.data.append((line[:-1], label))
        
#         print(len(self.data))
        self.data = []
        self.data_path = data_path
        with open(txt_path, 'r') as f:
            for line in f.readlines():
                data = line.split(',')
                self.data.append(data)

        print(len(self.data))

    def __getitem__(self, index):
        img, label = self.data[index][0], int(self.data[index][1])
        img = Image.open(os.path.join(self.data_path, img))

        img = img.convert('RGB')
        
        if self.is_train:
            data_transforms = transforms.Compose([
                transforms.Resize((256, 256)),
                transforms.RandomCrop(224),
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.RandomRotation(45),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ])

            img = data_transforms(img)
        else:
            data_transforms = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ])

            img = data_transforms(img)

        return img, label

    def __len__(self):
        return len(self.data)


class Trainer(object):
    def __init__(self, model, optimizer, train_loader, val_loader, out_path, max_iter):
        self.model = model
        self.opt = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.out_path = out_path
        self.max_iter = max_iter
        self.timestamp_start = datetime.datetime.now()

        if not os.path.exists(self.out_path):
            os.makedirs(self.out_path)

        self.log_train_headers = [
            'epoch',
            'iteration',
            'train/loss',
            'train/acc',
            'elapsed_time',
        ]

        self.log_val_headers = [
            'epoch',
            'iteration',
            'val/loss',
            'val/acc',
            'elapsed_time',
        ]

        if not os.path.exists(os.path.join(self.out_path, 'log_train.csv')):
            with open(os.path.join(self.out_path, 'log_train.csv'), 'w') as f:
                f.write(','.join(self.log_train_headers) + '\n')

        if not os.path.exists(os.path.join(self.out_path, 'log_val.csv')):
            with open(os.path.join(self.out_path, 'log_val.csv'), 'w') as f:
                f.write(','.join(self.log_val_headers) + '\n')

        self.epoch = 0
        self.iteration = 0

    def validate(self):
        training = self.model.training
        self.model.eval()

        val_loss = 0.
        acc_all = []

        for batch_idx, (img, label) in tqdm.tqdm(
                enumerate(self.val_loader),
                total=len(self.val_loader),
                desc='Validation Epoch=%d' % self.epoch,
                ncols=80,
                leave=False
        ):
            img, label = Variable(img), Variable(label)
            label = torch.tensor(label, dtype=torch.float32)
            img, label = img.cuda(), label.cuda()

            with torch.no_grad():
                result = self.model(img).cuda().squeeze(1)
#                 result = self.model(img).squeeze(1)

#             loss_fn = BCELoss(weight=None, reduce=True)
            loss_fn = CrossEntropyLoss(weight=None, reduce=True)
            loss = loss_fn(result, label.long())
            val_loss += loss

            acc = 0.
            label = label.cpu()
            result = result.cpu()
            for index, item in enumerate(result):
                if torch.max(item).item() == item[label.long()[index].item()].item():
                    acc += 1

            acc /= 16

            acc_all.append(acc)

        acc_all = np.array(acc_all)
        print('Val Acc=%s' % (str(acc_all.mean())))

        with open(os.path.join(self.out_path, 'log_val.csv'), 'a') as f:
            elapsed_time = (datetime.datetime.now() - self.timestamp_start).total_seconds()
            log = [self.epoch, self.iteration, val_loss.item(), acc_all.mean(), elapsed_time]
            log = map(str, log)
            f.write(','.join(log) + '\n')

        torch.save({
            'epoch': self.epoch,
            'iteration': self.iteration,
            'arch': self.model.__class__.__name__,
            'optim_state_dict': self.opt.state_dict(),
            'model_state_dict': self.model.state_dict(),
        }, os.path.join(self.out_path, 'checkpoint.pth.tar'))

        if training:
            self.model.train()

    def train_epoch(self):
        self.model.train()

        epoch_loss = 0.

        acc_all = []
        

        for batch_idx, (img, label) in tqdm.tqdm(
                enumerate(self.train_loader),
                total=len(self.train_loader),
                desc='Train Epoch=%d' % self.epoch,
                ncols=80,
                leave=False
        ):  
            iteration = batch_idx + self.epoch * len(self.train_loader)
            if self.iteration != 0 and (iteration - 1) != self.iteration:
                continue
            self.iteration = iteration
            self.opt.zero_grad()

            img, label = Variable(img), Variable(label)
            label = torch.tensor(label, dtype=torch.float32)
            img, label = img.cuda(), label.cuda()

            result = self.model(img).cuda().squeeze(1)
#             result = self.model(img).squeeze(1)

#             loss_fn = BCELoss(weight=None, reduce=True)
            loss_fn = CrossEntropyLoss(weight=None, reduce=True)
            loss = loss_fn(result, label.long())
            try:
                loss.backward()
                self.opt.step()
            except Exception as e:
                print(e)

            epoch_loss += loss.detach().cpu().numpy()

            if self.iteration > 0 and self.iteration % 3 == 0:
                acc = 0.
                label = label.cpu()
                result = result.cpu()
                for index, item in enumerate(result):
                    if torch.max(item).item() == item[label.long()[index].item()].item():
                        acc += 1
                    

                acc /= 16

                acc_all.append(acc)

                print('Train Acc=%s' % str(np.array(acc_all).mean()))

            if self.iteration >= self.max_iter:
                break

        with open(os.path.join(self.out_path, 'log_train.csv'), 'a') as f:
            elapsed_time = (datetime.datetime.now() - self.timestamp_start).total_seconds()
            log = [self.epoch, self.iteration, epoch_loss, np.array(acc_all).mean(), elapsed_time]
            log = map(str, log)
            f.write(','.join(log) + '\n')

    def train(self):
        max_epoch = int(math.ceil(1. * self.max_iter / len(self.train_loader)))
        for epoch in tqdm.trange(self.epoch, max_epoch, desc='Train', ncols=80):
            self.epoch = epoch
            self.train_epoch()
            self.validate()
            assert self.model.training


if __name__ == '__main__':
    drive.mount('/gdrive')
    
    data_transforms_train = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(45),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])

    data_transforms_val = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])

    train_set = datasets.ImageFolder(root='/gdrive/My Drive/style_classifier/style_data_multiclass', transform=data_transforms_train)

    val_set = datasets.ImageFolder(root='/gdrive/My Drive/style_classifier/style_data_multiclass_val', transform=data_transforms_val)

    train_loader = torch.utils.data.DataLoader(
        train_set,
        batch_size=16,
        shuffle=True
    )
    val_loader = torch.utils.data.DataLoader(
        val_set,
        batch_size=16,
        shuffle=True
    )

    model = StyleClassifier().cuda()

    opt = SGD(
        model.parameters(),
        lr=1e-4,
        momentum=0.1
    )

    trainer = Trainer(
        model=model,
        optimizer=opt,
        train_loader=train_loader,
        val_loader=val_loader,
        out_path='/gdrive/My Drive/style_classifier/log',
        max_iter=100000
    )

    trainer.epoch = 0
    trainer.iteration = 0
    trainer.train()


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth
Train Epoch=0:   0%|                                    | 0/224 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:245: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:46: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))

Train Epoch=0:   2%|▌                           | 4/224 [00:35<32:31,  8.87s/it]

Train Acc=0.1875



Train Epoch=0:   3%|▉                           | 7/224 [01:02<32:15,  8.92s/it]

Train Acc=0.15625


...

Train Acc=0.9354166666666667



Train Epoch=66:  41%|██████████▌               | 91/224 [01:14<01:45,  1.26it/s]

Train Acc=0.9354838709677419



Train Epoch=66:  42%|██████████▉               | 94/224 [01:16<01:43,  1.26it/s]

Train Acc=0.935546875
